In [80]:
import numpy as np
import time
import utils
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import model
import torch.optim as optim

In [81]:
batch_size = 1
output_folder = "output/" # folder path to save the results
save_results = True # save the results to output_folder
use_GPU = True # use GPU, False to use CPU
latent_size = 128 # bottleneck size of the Autoencoder model

In [82]:
from Dataloaders import GetDataLoaders

pc_array = np.load("data/chair_set.npy")
#pc_tensor = torch.load("preprocessed_point_cloud.pt")
#pc_tensor = torch.load("preprocessed_train_point_cloud.pt")
#pc_array = pc_tensor.numpy()
print(pc_array.shape)

# load dataset from numpy array and divide 90%-10% randomly for train and test sets
train_loader, test_loader = GetDataLoaders(npArray=pc_array, batch_size=batch_size, train_set_percentage=0.9)

# Assuming all models have the same size, get the point size from the first model
point_size = len(train_loader.dataset[0])
print(point_size)

(3746, 1024, 3)
1024


In [83]:
net = model.PointCloudAE(point_size,latent_size)

if(use_GPU):
    device = torch.device("mps")
    if torch.cuda.device_count() > 1: # if there are multiple GPUs use all
        net = torch.nn.DataParallel(net)
else:
    device = torch.device("cpu")

net = net.to(device)

In [84]:
#from pytorch3d.loss import chamfer_distance # chamfer distance for calculating point cloud distance
import torch
import numpy as np

def chamfer_distance(x, y, batch_reduction="mean", point_reduction="mean"):
    """
    Compute the chamfer distance between two point clouds x and y
    
    Parameters:
    -----------
    x: torch.Tensor
        First point cloud, shape (batch_size, num_points_x, dim)
    y: torch.Tensor
        Second point cloud, shape (batch_size, num_points_y, dim)
    batch_reduction: str
        Reduction operation to apply across the batch dimension: 'mean', 'sum' or None
    point_reduction: str
        Reduction operation to apply across the point dimension: 'mean', 'sum' or None
        
    Returns:
    --------
    dist_chamfer: torch.Tensor
        Chamfer distance between the point clouds
    """
    # Check input dimensions
    if x.dim() != 3 or y.dim() != 3:
        raise ValueError("Input point clouds must be 3-dimensional tensors")
    
    # Extract batch size and number of points
    batch_size, num_points_x, dim = x.shape
    _, num_points_y, _ = y.shape
    
    # Reshape to compute pairwise distances efficiently
    x_expanded = x.unsqueeze(2)  # [batch_size, num_points_x, 1, dim]
    y_expanded = y.unsqueeze(1)  # [batch_size, 1, num_points_y, dim]
    
    # Compute squared distances between each pair of points
    # |x-y|^2 = |x|^2 + |y|^2 - 2*x·y
    dist = ((x_expanded - y_expanded) ** 2).sum(dim=3)  # [batch_size, num_points_x, num_points_y]
    
    # For each point in x, find the distance to the closest point in y
    # Use values, not tuples from min()
    x_to_y = torch.min(dist, dim=2).values  # [batch_size, num_points_x]
    
    # For each point in y, find the distance to the closest point in x
    y_to_x = torch.min(dist, dim=1).values  # [batch_size, num_points_y]
    
    # Apply point reduction
    if point_reduction == "mean":
        x_to_y = x_to_y.mean(dim=1)  # [batch_size]
        y_to_x = y_to_x.mean(dim=1)  # [batch_size]
    elif point_reduction == "sum":
        x_to_y = x_to_y.sum(dim=1)  # [batch_size]
        y_to_x = y_to_x.sum(dim=1)  # [batch_size]
    elif point_reduction is None:
        pass
    else:
        raise ValueError(f"Invalid point_reduction: {point_reduction}")
    
    # Combine the two directional distances
    chamfer_dist = x_to_y + y_to_x  # [batch_size] or [batch_size, num_points]
    
    # Apply batch reduction
    if batch_reduction == "mean":
        chamfer_dist = chamfer_dist.mean()
    elif batch_reduction == "sum":
        chamfer_dist = chamfer_dist.sum()
    elif batch_reduction is None:
        pass
    else:
        raise ValueError(f"Invalid batch_reduction: {batch_reduction}")
    
    return chamfer_dist

optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [85]:
def train_epoch():
    epoch_loss = 0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        data = data.to(device)
        output = net(data.permute(0,2,1)) # transpose data for NumberxChannelxSize format
        loss = chamfer_distance(data, output) 
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss/i

In [86]:
def test_batch(data): # test with a batch of inputs
    with torch.no_grad():
        data = data.to(device)
        output = net(data.permute(0,2,1))
        loss = chamfer_distance(data, output)
        
    return loss.item(), output.cpu()

In [87]:
def test_epoch(): # test with all test set
    with torch.no_grad():
        epoch_loss = 0
        for i, data in enumerate(test_loader):
            loss, output = test_batch(data)
            epoch_loss += loss

    return epoch_loss/(i+1)

In [88]:
if(save_results):
    utils.clear_folder(output_folder)

In [89]:
# train_loss_list = []  
# test_loss_list = []  

# for i in range(1001) :

#     startTime = time.time()
    
#     train_loss = train_epoch() #train one epoch, get the average loss
#     train_loss_list.append(train_loss)
    
#     test_loss = test_epoch() # test with test set
#     test_loss_list.append(test_loss)
    
#     epoch_time = time.time() - startTime
    
#     writeString = "epoch " + str(i) + " train loss : " + str(train_loss) + " test loss : " + str(test_loss) + " epoch time : " + str(epoch_time) + "\n"
    
#     # plot train/test loss graph
#     plt.plot(train_loss_list, label="Train")
#     plt.plot(test_loss_list, label="Test")
#     plt.legend()

#     if(save_results): # save all outputs to the save folder

#         # write the text output to file
#         with open(output_folder + "prints.txt","a") as file: 
#             file.write(writeString)

#         # update the loss graph
#         plt.savefig(output_folder + "loss.png")
#         plt.close()

#         # save input/output as image file
#         if(i%50==0):
#             test_samples = next(iter(test_loader))
#             loss , test_output = test_batch(test_samples)
#             utils.plot_pointcloud_comparison(test_samples, test_output, show=False, save=True, name = (output_folder  + "epoch_" + str(i)))
#     else : # display all outputs
        
#         test_samples = next(iter(test_loader))
#         loss , test_output = test_batch(test_samples)
#         utils.plotPCbatch(test_samples,test_output)

#         print(writeString)

#         plt.show()
#         timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#         plt.savefig(f"model:{timestamp}.png")

        


In [90]:
import matplotlib.pyplot as plt
import os

# Training parameters
epochs = 100
learning_rate = 1e-4

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

for epoch in range(epochs):
    net.train()
    total_train_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs = data.to(device)
        optimizer.zero_grad()
        outputs = net(inputs.permute(0,2,1))
        loss, _, _ = chamfer_distance(inputs, outputs), inputs, outputs
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    net.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs = data.to(device)
            outputs = net(inputs.permute(0,2,1))
            loss, _, _ = chamfer_distance(inputs, outputs), inputs, outputs
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.6f} - Val Loss: {avg_val_loss:.6f}")

# Plot losses
plt.figure()
plt.plot(range(1, epochs+1), train_losses, label="Train Loss")
plt.plot(range(1, epochs+1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Chamfer Loss")
plt.legend()
plt.title("Training and Validation Loss")

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)
plot_path = os.path.join(output_folder, "loss_plot.png")
plt.savefig(plot_path)
plt.close()
print(f"Loss plot saved to: {plot_path}")


RuntimeError: Given groups=1, weight of size [64, 2, 1], expected input[1, 3, 1024] to have 2 channels, but got 3 channels instead